In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from tqdm import tqdm

In [3]:
data = pd.read_csv('Documentation/lab2_data.csv')

data_2 = pd.DataFrame(columns = ["Vehicle type", "Observed time", "Mean speed", "Accel time", "Decel time", "Idle time", "Cruise time"])

time_increment = 0.4

for _ID in tqdm(range(1,data["Vehicle_ID"].iloc[-1])):

    data_ID = data.loc[data["Vehicle_ID"] == _ID].reset_index() #Extract the rows with the ID
    measurement_lenght_ID = len(data_ID["Vehicle_ID"])
    speeds_ID = []
    accelerations_ID = []
    for i in range(measurement_lenght_ID - 1):
        speed_i_ID = (data_ID["Cumulative_distance"].iloc[i+1] - data_ID["Cumulative_distance"].iloc[i]) / time_increment
        if i < measurement_lenght_ID - 2:
            acceleration_i_ID = (data_ID["Cumulative_distance"].iloc[i+2] - 2 * data_ID["Cumulative_distance"].iloc[i+1] + data_ID["Cumulative_distance"].iloc[i]) / time_increment ** 2
        speeds_ID.append(speed_i_ID)
        accelerations_ID.append(acceleration_i_ID)
    
    vehicle_type_ID = data_ID["Vehicle_type"].iloc[0]

    observed_time_ID = (len(data_ID["Time"]) - 1) * time_increment

    mean_speed_ID = sum(speeds_ID)/len(speeds_ID)

    accel_time_ID = 0
    decel_time_ID = 0
    for accel_at_t in accelerations_ID:
        if accel_at_t < -0.5:
            decel_time_ID += time_increment
        elif accel_at_t > 0.5:
            accel_time_ID += time_increment
        else:
            pass

    idle_time_ID = 0
    cruise_time_ID = 0
    for speed_at_t in speeds_ID:
        if speed_at_t < 0.28:
            idle_time_ID += time_increment
        else:
            cruise_time_ID += time_increment

    data_2.loc[_ID] = [vehicle_type_ID, observed_time_ID, mean_speed_ID, accel_time_ID, decel_time_ID, idle_time_ID, cruise_time_ID]

data_2


100%|██████████| 2476/2476 [00:33<00:00, 72.97it/s] 


,Vehicle type,Observed time,Mean speed,Accel time,Decel time,Idle time,Cruise time
1,Motorcycle,65.6,1.969771,10.8,3.2,45.6,20.0
2,Car,32.0,6.759500,11.6,8.8,2.8,29.2
3,Car,39.2,5.688648,12.0,10.4,0.0,39.2
4,Motorcycle,99.2,1.469768,10.4,8.8,75.2,24.0
5,Motorcycle,93.6,2.374487,12.4,3.2,76.4,17.2
...,...,...,...,...,...,...,...
2472,Car,2.0,9.972000,0.8,1.2,0.0,2.0
2473,Car,2.0,7.357500,1.6,0.4,0.0,2.0
2474,Motorcycle,2.0,8.410000,0.0,0.8,0.0,2.0
2475,Motorcycle,2.0,11.352500,1.2,0.4,0.0,2.0


In [4]:
vehicle_types = ["Motorcycle", "Car", "Heavy Vehicle"]

data_per_vehicles = pd.DataFrame(columns = [
    "Vehicle type", 
    "Average speed [m/s]", 
    "Average speed [km/h]", 
    "Acceleration time [%]", 
    "Deceleration time [%]", 
    "Idle time [%]",
    "Cruise time [%]"])
index = 0

for vehicle_type in vehicle_types:
    index += 1
    data_vehicle_type = data_2.loc[data_2["Vehicle type"] == vehicle_type].reset_index()
    data_vehicle_type["Mean speed times observed time"] = data_vehicle_type["Observed time"] * data_vehicle_type["Mean speed"]

    total_time = data_vehicle_type["Observed time"].sum()
    mean_speed = data_vehicle_type["Mean speed times observed time"].sum() / total_time
    mean_speed_kmh = mean_speed * 3.6
    accel_time = data_vehicle_type["Accel time"].sum() / total_time * 100
    decel_time = data_vehicle_type["Decel time"].sum() / total_time * 100
    idle_time = data_vehicle_type["Idle time"].sum() / total_time * 100
    cruise_time = data_vehicle_type["Cruise time"].sum() / total_time * 100

    data_per_vehicles.loc[index] = [vehicle_type, mean_speed, mean_speed_kmh, accel_time, decel_time, idle_time, cruise_time]



In [5]:
data_per_vehicles

,Vehicle type,Average speed [m/s],Average speed [km/h],Acceleration time [%],Deceleration time [%],Idle time [%],Cruise time [%]
1,Motorcycle,5.259916,18.935698,22.002430,24.436717,34.456141,65.543859
2,Car,3.658174,13.169427,18.960666,20.088363,42.107098,57.892902
3,Heavy Vehicle,2.914529,10.492304,15.947461,16.979004,46.999900,53.000100
